<h1>Questions to be handed in on Newton's Method:</h1>

<p>Begin by loading our package for plotting and the Roots package</p>

In [ ]:
using Plots
gadfly()
using Roots

<hr />

<h3>Quick background</h3>

<p>Read about this material here: <a href="http://mth229.github.io/newton.html">Newton's Method</a>.</p>

<p>For the impatient, symbolic math – as is done behind the scenes at the Wolfram alpha web site – is pretty nice. For so many problems it can easily do what is tedious work. However, for some questions, only numeric solutions are possible. For example, there is no general formula to solve a fifth order polynomial the way there is a quadratic formula for solving quadratic polynomials. Even an innocuous polynomial like &#36;f&#40;x&#41; &#61; x^5 - x - 1&#36; has no easy algebraic solution:</p>

In [ ]:
using SymPy
@vars x

In [ ]:
using SymPy
x = symbols("x")

In [ ]:
solve(x^5 - x - 1)

<p>We see that <code>SymPy</code> basically punts on this question.</p>

<p>Numeric solutions are available. As this is a polynomial, we could use the <code>roots</code> function for <code>Roots</code>:</p>

In [ ]:
f(x) = x^5 - x - 1
roots(f)

5-element Array{Complex{Float64},1}:
    1.1673+0.0im     
  0.181232+1.08395im 
  0.181232-1.08395im 
 -0.764884+0.352472im
 -0.764884-0.352472im

<p>We see 5 roots – as expected from a fifth degree polynomial – with one real root (the one with <code>0.0im</code>) that is approximately 1.1673. Finding such a value usually requires some iterative root-finding algorithm (though not in the case above which uses linear algebra). For polynomials, the <code>fzeros</code> function uses such an algorithm <em>for polynomials</em> to find the real roots:</p>

In [ ]:
fzeros(f)                # no a, b range needed for polynomials.

1-element Array{Real,1}:
 1.1673

<p>Newton's method is a root-finding algorithm.  Like the bisection method discussed earlier, it is an <em>iterative algorithm</em>. The algorithm starts with some <em>guess</em> for a <em>zero</em> to an equation &#36;f&#40;x&#41;&#61;0&#36;. If this guess is called &#36;x_0&#36;, then the algorithm gives a <em>new (and improved!)</em> guess &#36;x_1&#36;. It is expected that &#36;x_1&#36; is a better guess, but may not be the best that can be. The algorithm is then repeated <em>again</em> to produce &#36;x_2&#36;. This is done until some guess, &#36;x_n&#36;, is as close enough or the algorithm fails for some reason. The <em>approximate zero</em> is taken to be &#36;x_n&#36;.</p>

<p>What is the algorithm for Newton's method? It is simple. If we start with some &#36;x_i&#36;, then &#36;x_&#123;i&#43;1&#125;&#36; is given by the intersection point of the &#36;x&#36;-axis of the tangent line of &#36;f&#40;x&#41;&#36; at &#36;x_i&#36;. Mathematically then we can equate our two methods to compute the slope of a tangent line:</p>

&#36;~
f&#39;&#40;x_i&#41; &#61; \frac&#123;f&#40;x_i&#41; - 0&#125;&#123;x_i - x_&#123;i&#43;1&#125;&#125;
~&#36;

<p>Or, solving for &#36;x_&#123;i&#43;1&#125;&#36;:</p>

&#36;~
x_&#123;i&#43;1&#125; &#61; x_i - f&#40;x_i&#41;/f&#39;&#40;x_i&#41;
~&#36;

<p>Let's see this algorithm for <code>f&#40;x&#41; &#61; x^3−2x−5</code>, a function that Newton himself considered. He was looking for a solution near &#36;2&#36;, so let's start there:</p>

In [ ]:
x = 2
f(x) = x^3 - 2x -5
fp(x) = 3x^2 - 2		# done by hand

fp (generic function with 1 method)

<p>We don't need to track the index (&#36;x_0&#36;, &#36;x_1&#36;, ...) as when we write the following expression, the next value is just assigned to <code>x</code> using the <em>current</em> value of <code>x</code> when computed:</p>

In [ ]:
x = x - f(x) / fp(x)
x, f(x)				# display both the new guess, x,  and the value f(x)

(2.1,0.06100000000000083)

<p>The value of &#36;2.1&#36; is a better guess, but not near the actual answer. We simply repeat to (hopefully) get a better guess:</p>

In [ ]:
x = x - f(x) / fp(x)
x, f(x)

(2.094568121104185,0.00018572317327247845)

<p>Here are a few more repeats:</p>

In [ ]:
x = x - f(x) / fp(x)
x, f(x)

(2.094551481698199,1.7397612239733462e-9)

In [ ]:
x = x - f(x) / fp(x)
x, f(x)

(2.0945514815423265,-8.881784197001252e-16)

<p>The value of <code>f&#40;x&#41;</code> is now <em>basically</em> 0, and any further updates to <code>x</code> do not change its value. We see that the algorithm has converged to an answer, <code>x</code>, and the fact that it is a zero is confirmed by the value of <code>f&#40;x&#41;</code>.</p>

<p>Repeating steps in <code>IJulia</code> can be a bit of a chore. There a few  ways to make this easier. For example, encapsulate the algorithm into a function or use a programming construct to repeat the task.</p>

<p>For the former, you might have:</p>

In [ ]:
newt(x, f, fp) = x - f(x)/fp(x)

newt (generic function with 1 method)

<p>and then for a given f, do something like</p>

In [ ]:
f(x) =  x^3 - 2x -5; fp(x) = 3x^2 - 2
newt(x) = newt(x, f, fp)
x = 2.0
newt(newt(newt(newt(newt(x)))))

2.0945514815423265

<p>That is kinda ugly. Here we use a programming construct, a <em>macro</em>, to repeat some _expression_ 5 times. (This macro basically replaces the expression internally with 5 repeats of the expression.)</p>

In [ ]:
macro take5(body) quote Float64[$(esc(body)) for _ in 1:5] end end # take5 macro

<p>Macros are prefaced with a <code>&#64;</code> in their name and are typically called without parentheses:</p>

In [ ]:
x = 2				# starting value
@take5     x = x - f(x) / fp(x)

5-element Array{Float64,1}:
 2.1    
 2.09457
 2.09455
 2.09455
 2.09455

<p>and to see that <code>x</code> has been updated we have:</p>

In [ ]:
x, f(x)

(2.0945514815423265,-8.881784197001252e-16)

<h3>Questions</h3>

<ul>
<li>Apply Newton's Method to the function &#36;f&#40;x&#41; &#61; \sin&#40;x&#41;&#36; with an   initial guess &#36;3&#36;. (This was historically used to compute many   digits of &#36;\pi&#36; efficiently.) What is the answer after 5 iterations?   What is the value of <code>sin</code> at the answer?</li>
</ul>

<p>The value of &#36;f&#40;x&#41;&#36; after 5 iterations is:</p>

<p>The value of &#36;f&#40;x&#41;&#61;\sin&#40;x&#41;&#36; at this approximate zero:</p>

<ul>
<li>Use Newton's method to find a zero for the function   &#36;f&#40;x&#41;&#61;x^5-x-1&#36;. Start at &#36;x&#61;1.6&#36;. What is the approximate root after   5 iterations? What is the value of &#36;f&#40;x&#41;&#36; for your answer? If you do   one or two more iterations, will your guess be better?</li>
</ul>

<p>The value after 5 iterations</p>

<p>The value of &#36;f&#40;x&#41;&#36;:</p>

<p>The value after two more iterations:</p>

<ul>
<li>Use Newton's method to find a zero of the function &#36;f&#40;x&#41; &#61; \cos&#40;x&#41; -   x&#36;. Make a graph to identify an initial guess.</li>
</ul>

<p>Show your commands below</p>

<p>The value of the approximate zero:</p>

<h3>Using <code>D</code> for the derivative</h3>

<p>If the function <code>f&#40;x&#41;</code> allows it, the <code>D</code> operator from the <code>Roots</code> package can simplify the Newton's method algorithm, as the derivative need not be computed by hand. In this case, the algorithm in <code>julia</code> becomes <code>x &#61; x - f&#40;x&#41;/D&#40;f&#41;&#40;x&#41;</code>.</p>

<ul>
<li>Use Newton's method to find an intersection point of &#36;f&#40;x&#41; &#61;   e^&#123;-x^2&#125;&#36; and &#36;g&#40;x&#41;&#61;x&#36;. (Look at &#36;h&#40;x&#41; &#61; f&#40;x&#41; - g&#40;x&#41; &#61; 0&#36;.) Start   with a guess of &#36;0&#36;.</li>
</ul>

<ul>
<li>Use Newton's method to find <em>both</em> positive intersection points of   &#36;f&#40;x&#41; &#61; e^x&#36; and &#36;g&#40;x&#41; &#61; 2x^2&#36;. Make a graph to identify good   initial guesses.</li>
</ul>

<p>The smallest value is:</p>

<p>The largest value is:</p>

<h3>Using <code>newton</code> and <code>fzero</code> from the <code>Roots</code> package</h3>

<p>The <code>newton</code> function in the <code>Roots</code> package will compute newton's method. For example:</p>

In [ ]:
f(x) = sin(x)
fp(x) = cos(x)
x = 3
newton(f, fp, x)

3.141592653589793

<p>The extra argument <code>verbose&#61;true</code> will show the iterations:</p>

In [ ]:
newton(f, fp, 3, verbose=true)

3.141592653589793

<p>However, the <code>fzero</code> function – that we have seen before – will use a derivative-free algorithm, similar to Newton's method to find a zero. So, the above zero can also be found with:</p>

In [ ]:
fzero(sin, 3)

3.141592653589793

<p>(That is right, <code>fzero</code> can be used two different ways – at least. Above it is called with an initial guess. Previously, we called it with a bracketing interval, as in <code>fzero&#40;sin, &#91;3,4&#93;&#41;</code>. If you specify a bracketing interval, <code>fzero</code> will use an algorithm guaranteed to converge. If you just specify an initial guess, the convergence is generally faster, but may not happen.)</p>

<hr />

<ul>
<li>find a zero of &#36;f&#40;x&#41; &#61; x\cdot &#40;2&#43;\ln&#40;x&#41;&#41;&#36; starting at &#36;1&#36;. What is   your answer? How small is the function for this value?</li>
</ul>

<p>What is the value of the zero?</p>

<p>The value of the function at the zero?</p>

<ul>
<li>Use <code>fzero</code> to find all zeros of the function &#36;f&#40;x&#41; &#61; 2\sin&#40;x&#41; -   \cos&#40;2x&#41;&#36; in &#36;&#91;0, 2\pi&#93;&#36;. (Graph first to see approximate answers.)</li>
</ul>

<ul>
<li>Use <code>fzero</code> to find when the derivative of &#36;f&#40;x&#41; &#61; 5/\cos&#40;x&#41; &#43;   7/\sin&#40;x&#41;&#36; is &#36;0&#36; in the interval &#36;&#40;0, \pi/2&#41;&#36;.</li>
</ul>

<h3>When Newton's method fails</h3>

<p>Newton's method can fail due to various cases:</p>

<ul>
<li>the initial guess is not close to the zero</li>
</ul>

<ul>
<li>the derivative, &#36;|f&#39;&#40;x&#41;|&#36; is too small</li>
</ul>

<ul>
<li>the second derivative &#36;|f&#39;&#39;&#40;x&#41;|&#36; is too big, or possibly undefined</li>
</ul>

<ul>
<li>Earlier  the roots of &#36;f&#40;x&#41; &#61; x^5 - x - 1&#36; were considered. Try   Newton's method with an initial guess of &#36;x_0&#61;0&#36; to find a real   root. Why does this fail? (You can look graphically. Otherwise, you   could look at the output of <code>newton</code> with this extra argument:   <code>newton&#40;f, fp, x0, verbose&#61;true&#41;</code>.</li>
</ul>

<ul>
<li>Let <code>f&#40;x&#41; &#61; abs&#40;x&#41;^&#40;1/3&#41;</code>. Starting at <code>x&#61;1</code>, Newton's method will   fail to converge. What happens? Are any of the above 3 reason's to   blame?</li>
</ul>

<h3>Quadratic convergence</h3>

<p>When Newton's method converges to a <em>simple zero</em> it is said to have <em>quadratic convergence</em>. A simple zero is one with multiplicity 1 and quadratic convergence says basically that the error at the &#36;i&#43;1&#36;st step is like the error for &#36;i&#36;th step squared. In particular, if the error is like &#36;10^&#123;-3&#125;&#36; on one step, it will be like &#36;10^&#123;-6&#125;&#36;, then &#36;10^&#123;-12&#125;&#36; then &#36;10^&#123;-24&#125;&#36; on subsequent steps. (Which is typically beyond the limit of a floating point approximation.) This is why one can <em>usually</em> take just 5, or so, steps to get to an answer.</p>

<p>Not so for multiple roots. </p>

<ul>
<li>For the function <code>f&#40;x&#41; &#61; &#40;8x*exp&#40;-x^2&#41; -2x - 3&#41;^8</code>, starting with   <code>x&#61;-2.0</code>, Newton's method will converge, but it will take many steps   to get to an answer that has &#36;f&#40;x&#41;&#36; around &#36;10^&#123;-16&#125;&#36;. How many?   Roughly how many iterations do you need? (A single call of    <code>&#64;take5 x &#61; x-f&#40;x&#41;/D&#40;f&#41;&#40;x&#41;</code> gives an answer with <code>f&#40;x&#41; &#61; 0.00028</code> only.)</li>
</ul>

<ul>
<li>Repeat the above with <code>f&#40;x&#41; &#61; 8x*exp&#40;-x^2&#41; -2x - 3</code> – there is no   extra power of &#36;8&#36; here – and again, starting with   <code>x&#61;-2.0</code>. Roughly how many iterations are needed now?</li>
</ul>